In [30]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file = "the-verdict.txt"
urllib.request.urlretrieve(url, file)

('the-verdict.txt', <http.client.HTTPMessage at 0x1b0352ebdd0>)

In [31]:
with open (file , 'r' , encoding = 'utf-8') as f :
    raw = f.read()
print("The raw file is: ", len(raw))
print(raw[:99])

The raw file is:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [32]:
import re
text  = "Hello, world. This, is a test."
res = re.split(r'(\s)',text)
print (res)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [33]:
res = re.split(r'([,.]|\s)',text)
print (res)
res = [item for item in res if item.strip()]
print(res)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']
['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [34]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)


['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [35]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed) , preprocessed[:100])

4690 ['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself', 'in', 'a', 'villa', 'on', 'the', 'Riviera', '.', '(', 'Though', 'I', 'rather', 'thought', 'it', 'would', 'have', 'been', 'Rome', 'or', 'Florence', '.', ')', '"', 'The', 'height', 'of', 'his', 'glory', '"', '--', 'that', 'was', 'what', 'the', 'women', 'called', 'it', '.', 'I', 'can', 'hear', 'Mrs', '.', 'Gideon', 'Thwing', '--', 'his', 'last', 'Chicago', 'sitter', '--']


In [36]:
 all_words = sorted(set(preprocessed))
 vocab_size = len(all_words)
 print(vocab_size)

1130


In [37]:
vocab = {token: integer for integer, token in enumerate(all_words)}
for i , item in enumerate(vocab.items()):
    print(item)
    if i>50 :
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


In [38]:
class simpleTokenizerV1:
    def __init__(self , vocab):
        self.vocab = vocab
        self.str_to_int = self.vocab
        self.int_to_str = {i: s for s,i in vocab.items()}
    def encode(self, text):
        txt = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        txt = [item.strip() for item in txt if item.strip()]
        ids = [self.str_to_int[s] for s in txt]
        return ids
    def decode(self, ids):
        txt = " ".join([self.int_to_str[id] for id in ids])
        txt =  re.sub(r'\s+([,.?!"()\'])', r'\1', txt) 
        return txt
    
        

In [39]:
tokenizer = simpleTokenizerV1(vocab)
text =  """It's the last he painted, you know," 
       Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [40]:
all_token = sorted(list(set(preprocessed)))
all_token.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token: i for i , token in enumerate(all_token)}
print(len(vocab))

1132


In [41]:
for i , item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [42]:
class simpleTokenizerV2:
    def __init__(self , vocab):
        self.vocab = vocab
        self.str_to_int = self.vocab
        self.int_to_str = {i: s for s,i in vocab.items()}
    def encode(self, text):
        txt = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        txt = [item.strip() for item in txt if item.strip()]
        txt = [item if item in self.str_to_int else "<|unk|>" for item in txt]
        ids = [self.str_to_int[s] for s in txt]
        return ids
    def decode(self, ids):
        txt = " ".join([self.int_to_str[id] for id in ids])
        txt =  re.sub(r'\s+([,.?!"()\'])', r'\1', txt) 
        return txt
    
        

In [43]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [44]:
tokenizer = simpleTokenizerV2(vocab)
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))


[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [45]:
# pip install tiktoken

In [46]:
from importlib.metadata import version
import tiktoken
__version__ = version('tiktoken')
print(__version__)

0.8.0


In [47]:
tokenizer = tiktoken.get_encoding("gpt2")

In [48]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace. pranjal sgkjnoi!@#$%^&*()"
 )
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
strings = tokenizer.decode(integers)
print(strings)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13, 778, 272, 73, 282, 264, 70, 42421, 3919, 72, 0, 31, 29953, 4, 61, 5, 9, 3419]
Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace. pranjal sgkjnoi!@#$%^&*()


In [49]:
text = (
    "Akwirw ier"
 )
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


In [50]:
 with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
 enc_text = tokenizer.encode(raw_text)
 print(len(enc_text))

5145


In [51]:
enc_sample = enc_text[50:]
# print(enc_sample)

In [52]:
context_size = 4        
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [53]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [54]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [56]:
import torch 
from torch.utils.data import Dataset , DataLoader

In [57]:
class GPTDatasetV1(Dataset):
    def __init__(self , txt , tokenizer , max_len , stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)
        for i in range(0,len(token_ids)-max_len , stride):
            input_chunk = token_ids[i:i + max_len]
            target_chunk = token_ids[i + 1: i + max_len + 1]
            self.input_ids.append(input_chunk)
            self.target_ids.append(target_chunk)
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
            
        

In [86]:
def create_dataloaderv1(txt , batchsize  = 4 ,  max_len = 256 , stride = 128 , shuffle = True , drop_last = True , num_workers = 0):
    tokenizer =  tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt , tokenizer, max_len , stride)
    dataloader =  dataloader = DataLoader(
        dataset,
        batch_size=batchsize,
        shuffle=shuffle,
        drop_last=drop_last,    
        num_workers=num_workers    
    )
    return dataloader

In [87]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
dataloader = create_dataloaderv1(
    raw_text, batchsize=1, max_len=4, stride=1, shuffle=False , )
data_iter = iter(dataloader)
print(next(data_iter))

[[tensor([40]), tensor([367]), tensor([2885]), tensor([1464])], [tensor([367]), tensor([2885]), tensor([1464]), tensor([1807])]]


In [88]:
print(next(data_iter))

[[tensor([367]), tensor([2885]), tensor([1464]), tensor([1807])], [tensor([2885]), tensor([1464]), tensor([1807]), tensor([3619])]]


In [89]:
 dataloader = create_dataloaderv1(
    raw_text, batchsize=8, max_len=4, stride=4,
    shuffle=False
 )
 data_iter = iter(dataloader)
 inputs, targets = next(data_iter)
 print("Inputs:\n", inputs)
 print("\nTargets:\n", targets)

Inputs:
 [tensor([   40,  1807, 10899, 15632,   922,   568,  1049,   284]), tensor([ 367, 3619, 2138,  438, 5891,  340, 5975, 3285]), tensor([2885,  402,  257, 2016, 1576,  373,  284,  326]), tensor([1464,  271, 7026,  257,  438,  645,  502,   11])]

Targets:
 [tensor([ 367, 3619, 2138,  438, 5891,  340, 5975, 3285]), tensor([2885,  402,  257, 2016, 1576,  373,  284,  326]), tensor([1464,  271, 7026,  257,  438,  645,  502,   11]), tensor([ 1807, 10899, 15632,   922,   568,  1049,   284,   287])]


In [91]:
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [92]:
print(embedding_layer.weight.shape)
print(embedding_layer(input_ids))

torch.Size([6, 3])
tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [93]:
 vocab_size = 50257
 output_dim = 256
 token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [104]:
 max_length = 4
 dataloader = create_dataloaderv1(
    raw_text, batchsize=8, max_len=max_length,
   stride=max_length, shuffle=False
 )
 data_iter = iter(dataloader)
 inputs, targets = next(data_iter)
 print("Token IDs:\n", inputs)
 print("\nInputs shape:\n", len(inputs) , len(inputs[0]))

Token IDs:
 [tensor([   40,  1807, 10899, 15632,   922,   568,  1049,   284]), tensor([ 367, 3619, 2138,  438, 5891,  340, 5975, 3285]), tensor([2885,  402,  257, 2016, 1576,  373,  284,  326]), tensor([1464,  271, 7026,  257,  438,  645,  502,   11])]

Inputs shape:
 4 8


In [107]:
inputs = torch.stack(inputs)
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)


torch.Size([4, 8, 256])


In [108]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])
